In [18]:
import opensmile 
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import random
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn import svm
from pprint import pprint
base_path = "/home/ubuntu/"

## Read saved dataframes to generate features and labels

In [5]:
data_path_lj = base_path + 'testing-code/opensmile-feature-importance/smile_dfs/LJ_sample_11200.csv'
df = pd.read_csv(data_path_lj)

In [6]:
df.head()

,file,start,end,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,...,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope,label
0,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0 days,0 days 00:00:06.531927438,2.255606,0.009317,0.214286,0.246159,0.494542,0.820717,0.248383,...,0.620289,2.236345,2.219888,19.675816,0.517250,105.152430,48.318115,99.288990,46.019333,0
1,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0 days,0 days 00:00:08.250204082,2.478899,0.688725,0.093137,0.349024,0.710767,1.087815,0.361742,...,0.546407,3.298283,3.298329,-20.000000,0.466784,126.802810,60.297080,111.975380,78.029175,0
2,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0 days,0 days 00:00:05.777278912,2.404334,0.089631,0.562390,0.361210,0.625174,1.103133,0.263965,...,0.426229,3.179683,3.202073,-19.708340,0.641415,126.600235,65.582650,133.139340,72.428530,0
3,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0 days,0 days 00:00:07.785804989,1.841870,0.856957,0.998700,0.396953,0.643860,0.954928,0.246908,...,0.639111,2.808894,2.806713,19.999980,0.459220,128.068300,59.187996,113.987816,61.324000,0
4,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0 days,0 days 00:00:07.124036281,2.283360,0.034139,0.022760,0.400029,0.714020,1.130978,0.313991,...,0.564978,2.923608,2.915823,19.985918,0.558662,123.728270,74.657330,120.433530,64.156030,0


In [7]:
y = df['label'].copy()
X = df.drop(columns=['file', 'start', 'end','label']).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
#standard scaling of input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
#robust scaling of input data
scaler_ro = RobustScaler()
X_train_scaled_ro = scaler_ro.fit_transform(X_train)
X_test_scaled_ro = scaler_ro.transform(X_test)

In [12]:
#min-max scaling of data
scaler_mm = MinMaxScaler()
X_train_scaled_mm = scaler_mm.fit_transform(X_train)
X_test_scaled_mm = scaler_mm.transform(X_test)

## Experiment 1: Random Forest Classifier based Feature Importance

In [25]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]

# Number of features to consider at every split
max_features = ['log2', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 50]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 5, 10]

# Method of selecting samples for training each tree
bootstrap = [True, False]

#criterion 
criterion = ['gini', 'entropy', 'log_loss']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['log2', 'sqrt'],
 'min_samples_leaf': [1, 5, 10],
 'min_samples_split': [2, 5, 10, 50],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [27]:
rf = RandomForestClassifier()

In [28]:
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 5, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)

In [30]:
rf_random.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=900; total time=  43.9s
[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=700; total time=  33.7s
[CV] END bootstrap=True, max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=50, n_estimators=900; total time=  47.1s
[CV] END bootstrap=True, max_depth=100, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=  43.2s
[CV] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=  27.9s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time= 6.7min
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimato

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1, 5, 10],
                                        'min_samples_split': [2, 5, 10, 50],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, verbose=2)

In [31]:
rf_random.best_params_

{'n_estimators': 900,
 'min_samples_split': 50,
 'min_samples_leaf': 10,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': False}

In [32]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [33]:
#hyperparameter selection
n_estimators = 500 #default=100
criterion = "gini" #{“gini”, “entropy”, “log_loss”}, default=”gini”
max_depth = 8 #default=None 
min_samples_split = 2 #default=2 The minimum number of samples required to split an internal node
min_samples_leaf = 1 #default=1 The minimum number of samples required to be at a leaf node.
min_weight_fraction_leaf = 0.0 #The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node.
max_features = "log2" #default="sqrt" The number of features to consider when looking for the best split:
max_leaf_nodes = None #Grow trees with max_leaf_nodes in best-first fashion.
min_impurity_decrease = 0.0 #A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
bootstrap = True #default = True Whether bootstrap samples are used when building trees.
oob_score = True #default = False Whether to use out-of-bag samples to estimate the generalization score.
n_jobs = -1 #default=None -1 means all processors
random_state = None #deafult=None controls randomness of bootstrapping
verbose = 0 
warm_start = False #deafult = False When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble, otherwise, just fit a whole new forest.
max_samples = None #If bootstrap is True, the number of samples to draw from X to train each base estimator.

In [34]:
#define model
base_model = RandomForestClassifier(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    criterion=criterion,
                                    max_features=max_features,
                                    bootstrap=bootstrap,
                                    oob_score=oob_score,
                                    n_jobs=n_jobs)

base_model.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=8, max_features='log2', n_estimators=500,
                       n_jobs=-1, oob_score=True)

In [36]:
base_preds = base_model.predict(X_test_scaled)
base_accuracy = accuracy_score(y_test,preds)

In [38]:
base_accuracy*100

77.76785714285714

In [39]:
best_random = rf_random.best_estimator_
random_preds = best_random.predict(X_test_scaled)
random_accuracy = accuracy_score(y_test, random_preds)

In [40]:
random_accuracy*100

86.29464285714286